### Table of Contents

# 1. Import Data

## 1.1 Import the needed libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from datetime import date

%matplotlib inline
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

## 1.2 Import a integrate data

In [ ]:
df_crm = pd.read_csv('crm.csv')
df_mkt = pd.read_csv('mkt.csv')
df_sales = pd.read_excel('sales.xlsx')

In [ ]:
df = pd.merge(pd.merge(df_crm,df_sales,on='CustomerID',how="inner"),df_mkt,on="CustomerID",how="inner")

## 1.3 Set Index


In [ ]:
df.set_index('CustomerID',inplace = True)

## 1.4 Check and removing duplicates

In [ ]:
df[df.duplicated()] # checking duplicates

In [ ]:
df = df[~df.duplicated()] # drop duplicates rows

# 2. Explore Data

## 2.0 Data profiling

Se não quiserem instalar a biblioteca não corram esta secção. Caso contrário o comando para instalar é pip install ydata-profiling. No final **apagar esta secção**

In [ ]:
#from ydata_profiling import ProfileReport
#profile= ProfileReport (df, title= "DSML_Project")

In [ ]:
#profile.to_file('DSML_profile.html')

## 2.1 Basic Exploration

Q: _To check the number of columns and rows_ we used `shape` _attribute_

In [ ]:
df.shape

> A: _The dataset has **7000 rows** and **26 columns**_

__*Q*__: Check the name of the features of the dataset we used `columns` _attribute_

In [ ]:
df.columns

> A: The dataset has the following columns/features names: <br>
        >Index. CustomerID
        >1. 'Name' <br>
        >2. 'Birthyear'<br>
        >3. 'Education'<br>
        >4. 'Marital_Status'<br>
        >5. 'Income'<br>
        >6. 'Kid_Younger6'<br>
        >7. 'Children_6to18'<br>
        >8. 'Date_Adherence'<br>
        >9. 'Recency'<br>
        >10. 'MntMeat&Fish'<br>
        >11. 'MntEntries'<br>
        >12. 'MntVegan&Vegetarian'<br>
        >13. 'MntDrinks'<br>
        >14. 'MntDesserts'<br>
        >15. 'MntAdditionalRequests'<br>
        >16. 'NumOfferPurchases'<br>
        >17. 'NumAppPurchases'<br>
        >18. 'NumTakeAwayPurchases'<br>
        >19. 'NumStorePurchases'<br>
        >20. 'NumAppVisitsMonth'<br>
        >21. 'Complain'<br>
        >22. 'Response_Cmp1'<br>
        >23. 'Response_Cmp2'<br>
        >24. 'Response_Cmp3'<br>
        >25. 'Response_Cmp4'<br>
        >26. 'Response_Cmp5'<br>

Q: First glance of the dataset using `head` and `tail` methods to check the first and last 5 rows.

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

Q: To check the basic information of the dataset we've used the `info` method

In [ ]:
df.info()

>A: We can observe the data type of the dataset and the how many of features per data type  `dtypes: float64 - (7), int64 - (15), object - (4)`, the memory usage of `1.4+MB`, and the non-null values present per columns. <br>
> Using only `info` method we understand that `'Education', 'Recency', 'MntDrinks'` have __14, 23, 28 null values__ that require some action.

# 2.2 Statistical Exploration

## 2.2.1 Numerical Variables

In [ ]:
df.describe()

> _The describe return we can get a first glance and make some conclusion:_

>__Birthyear__ - could originate an Age column for readability purposes<br>
__Income__ - Min and Max are very far from each other and far from the mean value which could indicate outliers<br>
__Recency__ - 6977 valid values, hence we should look in deep and decide on how to minimize that effect of missing values<br>
__MntMeat&Fish__ - Min and Max are distant from each other and have high standard deviation which could effect some future conclusion<br>
__MntEntries__ - Again has high standard deviation that we should analyze, Min and Max far apart, similar to MntMeat&Fish<br>
__MntVegan&Vegetarian__ - Similar to the previous two Mnt columns<br>
__MntDrinks, MntDesserts__ - Seems to be very similar between them<br>
__MntAdditionalRequests__ - The max value standard deviation seems high and also the max value very far apart from the mean<br>
__NumOfferPurchases, NumTakeAwayPurchases, NumAppVisitsMonth__  - Have a max value to distante from the mean that could be true but we need to take into account<br>
__NumAppPurchases, SumStorePurchases__ - Seems does not have strange summary statistcs<br>
__Kid_Younger6, Children_6to18__ - 75% of clients have at least one child

**Q**: Skewness of each variable 

In [ ]:
df.skew()

Concerning the variables' skewness, we can conclude the following:
- `Moderate skewness (between |0.5| and |1.0|)`: Birthyear, Income, Kid_Younger6, Children_6to18, Recency, NumAppPurchases, NumStorePurchases, NumAppVisitsMonth
- `High skewness (higher than |1.0|)`: MntMeat&Fish, MntEntries, MntVegan&Vegetarian, MntDrinks, MntDesserts, MntAdditionalRequests, NumOfferPurchases, NumTakeAwayPurchases, Complain, Response_Cmp1, Response_Cmp2, Response_Cmp3, Response_Cmp4, Response_Cmp5

In [ ]:
df.kurt()

Features with kurtosis higher than 3 could indicate presence of outliers, hence we should have special considerantion with the following features:
>MntEntries, MntVegan&Vegetarian, MntDrinks, MntDesserts, NumOfferPurchases, NumAppVisitsMonth

Note: Binomial Variables Complain, and Response_Cmp1 the kurtosis we will not consider as outliers

## 2.2.2 Categorical Values

In [ ]:
df.describe(include = object)

> We can conclude that the education as **14 missing** values

#### Level/Possible values of Categorical Features

### `Name` prefix unique values and count

In [ ]:
df['Name'].str.partition(" ")[0].value_counts()

With the prefix we can generate a `gender` feature to further explore the dataset. We will deal with that in the data transformation capther

#### **`Gender`** feature creation

In [ ]:
df["Gender"] = df['Name'].str.partition(" ")[0]
df = df.replace({"Gender":{"Mr.": 1,"Miss": 0,"Mrs.": 0}})

### `Education` unique values and count

In [ ]:
df["Education"].value_counts()

We have some issues that will need trasformatioin:<br>
- Graduation, Master, HighSchool are written in different ways<br>
- `Basic` and `HighSchool` need different levels?

#### Education standardization

In [ ]:
df = df.replace({"Education":{"master":"Master", "graduation":"Graduation", "phd":"PhD","highschool":"HighSchool"}})

### `Marital_Status` unique values and count

In [ ]:
df["Marital_Status"].value_counts()

Similarly to previous feature we also have some issues that need transformation:<br>
- Married, Together, Single, Divorced and Widow are written with lower and capital letters
- We could also consider that Married and Together are similar and joined them in the same level<br>

#### Marital_Status standardization

In [ ]:
df = df.replace({"Marital_Status":{"married":"Married", "together":"Married", "single":"Single","widow":"Widow","divorced":"Divorced","Together":"Married"}})
df["Marital_Status"].value_counts()

`Date_Adherence` unqiue values and count

In [ ]:
df["Date_Adherence"].value_counts()

`Date_Adherence` is a date and will need transformation to a date format for further exploration

## 2.3 Visual Exploration

### 2.3.1 Numerical Variables

## 2.4. In-Depth Exploration

# 3. Preprocess Data

## 3.1. Data Cleaning

### 3.1.1. Outliers

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
#ax1.boxplot(df['MntVegan&Vegetarian'])
#ax2.boxplot(df['Income'])

In [ ]:
df.drop(df[abs(zscore(df['MntVegan&Vegetarian'])) > 3].index, inplace=True)
df.drop(df[abs(zscore(df['Income'])) > 3].index, inplace=True)

In [ ]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
#ax1.boxplot(df['MntVegan&Vegetarian'])
#ax2.boxplot(df['Income'])

## 3.1.2 Skewness Correction

In [ ]:
df['MntMeat&Fish'] = df['MntMeat&Fish'].apply(lambda x: np.log10(x+1))
df['MntVegan&Vegetarian'] = df['MntVegan&Vegetarian'].apply(lambda x: np.log10(x+1))
df['MntEntries'] = df['MntEntries'].apply(lambda x: np.log10(x+1))
df['MntDrinks'] = df['MntDrinks'].apply(lambda x: np.log10(x+1))
df['MntDesserts'] = df['MntDesserts'].apply(lambda x: np.log10(x+1))
df['MntAdditionalRequests'] = df['MntAdditionalRequests'].apply(lambda x: np.log10(x+1))
df['NumOfferPurchases'] = df['NumOfferPurchases'].apply(lambda x: np.log10(x+1))
df['NumTakeAwayPurchases'] = df['NumTakeAwayPurchases'].apply(lambda x: np.log10(x+1))
df['Complain'] = df['Complain'].apply(lambda x: np.log10(x+1))
df['Response_Cmp1'] = df['Response_Cmp1'].apply(lambda x: np.log10(x+1))
df['Response_Cmp2'] = df['Response_Cmp2'].apply(lambda x: np.log10(x+1))
df['Response_Cmp3'] = df['Response_Cmp3'].apply(lambda x: np.log10(x+1))
df['Response_Cmp4'] = df['Response_Cmp4'].apply(lambda x: np.log10(x+1))
df['Response_Cmp5'] = df['Response_Cmp5'].apply(lambda x: np.log10(x+1))

### 3.1.2. Missing Values

In [ ]:
Response_is_null = df["Response_Cmp1"].isna().sum() + df["Response_Cmp2"].isna().sum() + df["Response_Cmp3"].isna().sum() + df["Response_Cmp4"].isna().sum()
Response_is_null == 0

In [ ]:
df.info()

In [ ]:
df.isna().sum()

- **`Education`**, **`Recency`**, **`MntDrinks`** and **`MntTotal`** (due to dependancy of `MntDrinks`) have missing values

#### Filling the missing values

Fill `Education` with the mode

In [ ]:
df["Education"].fillna(df["Education"].mode()[0], inplace = True)

Fill `Recency` with the median value

In [ ]:
df["Recency"].fillna(df["Recency"].mean(), inplace = True)

In [ ]:
#df.drop(columns = "MntTotal", inplace = True)

In [ ]:
df_mnt = df[[ 'MntMeat&Fish', 'MntEntries', 'MntVegan&Vegetarian', 'MntDrinks',
       'MntDesserts', 'MntAdditionalRequests']]

imputer = KNNImputer(n_neighbors=3)
array_impute = imputer.fit_transform(df_mnt)
df_mnt = pd.DataFrame(array_impute, columns = df_mnt.columns)

In [ ]:
df["MntDrinks"] = df_mnt["MntDrinks"].values

In [ ]:
df["MntTotal"] = df['MntMeat&Fish'] + df['MntEntries'] + df['MntVegan&Vegetarian'] + df['MntDrinks'] + df['MntDesserts']

In [ ]:
df.isna().sum()

## 3.2. Data Transformation

### 3.2.1. Create new Variables

### Utils

#### Creating Age variable from the Birthyear

In [ ]:
df['Age'] = df.Birthyear.apply(lambda x: date.today().year-x)

In [ ]:
df.drop('Birthyear', axis= 1, inplace = True)

#### Creating card adherence age variable from the Date adherence

In [ ]:
from datetime import datetime
df = df.replace({"Date_Adherence":{"2/29/2022": datetime.strptime("2022-03-01", '%Y-%m-%d')}}) #2022 is not a leap year, therefore 29/02/2022 is not a possible day

In [ ]:
df['daysAsCardClient'] = df['Date_Adherence'].apply(lambda x: (date.today() - x.date()).days)

In [ ]:
df.drop('Date_Adherence', axis= 1, inplace = True)

#### Fill Education

In [ ]:
edu_encode = pd.get_dummies(df.Education, drop_first= True)
df = pd.concat([df, edu_encode], axis = 1)

In [ ]:
df.drop('Education', axis = 1, inplace = True)

#### Fill Maritial Status

In [ ]:
marital_encode = pd.get_dummies(df.Marital_Status, drop_first= True)
df = pd.concat([df, marital_encode], axis = 1)

In [ ]:
df.drop('Marital_Status', axis= 1, inplace = True)

#### Create MntTotal

In [ ]:
df["MntTotal"] = df['MntMeat&Fish'] + df['MntEntries'] + df['MntVegan&Vegetarian'] + df['MntDrinks'] + df['MntDesserts'] + df['MntAdditionalRequests']
df["MntTotal"]
# em falta Mnt Add Requests

#### Create Mnt Pday Card

In [ ]:
df['Mnt_pday_card']= df.MntTotal/df.daysAsCardClient

#### Create Response Campaigns

In [ ]:
df['Response_Campaigns'] = df['Response_Cmp1'] + df['Response_Cmp2'] + df['Response_Cmp3'] + df['Response_Cmp4'] + df[
       'Response_Cmp5']

In [ ]:
# queremos dar drop nos Response, tudo certo ok? pq?
df.drop(['Response_Cmp1', 'Response_Cmp2', 'Response_Cmp3', 'Response_Cmp4', 'Response_Cmp5'], axis=1, inplace=True)

#### Create Total Kids

In [ ]:
df["Total_Kids"] = df["Kid_Younger6"] + df["Children_6to18"]

#### Create Has Kids

In [ ]:
df["has_Kids"] = df["Total_Kids"].apply(lambda x: 0 if x == 0 else 1)
df["has_Kids"]

In [ ]:
# demos drop pq?
df.drop(['Kid_Younger6', 'Children_6to18'], axis = 1, inplace = True)

#### Create age_bins

In [ ]:
df["age_bins"] = pd.cut(df["Age"], bins = 5)
age_bin = pd.get_dummies(df['age_bins'],prefix='age')
df = pd.concat([df,age_bin], axis=1)
df.drop(['age_bins'],axis=1, inplace=True)

In [ ]:
df

## Incoherencies


In [ ]:
df[(df[['MntMeat&Fish', 'MntEntries',
        'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
        'MntAdditionalRequests']].sum(axis = 1) > 0) & (df[['NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0)]

In [ ]:
# como justificar a atribuição de [0,0,0,0,0,0] às variaveis de monetary?

df.loc[(df[['MntMeat&Fish', 'MntEntries',
            'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
            'MntAdditionalRequests']].sum(axis = 1) > 0) & (df[['NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0),['MntMeat&Fish','MntEntries','MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts','MntAdditionalRequests']] = [0,0,0,0,0,0]

In [ ]:
df[(df[['MntMeat&Fish', 'MntEntries',
        'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
        'MntAdditionalRequests']].sum(axis = 1) > 0) & (df[['NumAppPurchases', 'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0)] # confirmação do ajuste

In [ ]:
df[(df['NumOfferPurchases'] > df[['NumAppPurchases','NumTakeAwayPurchases','NumStorePurchases']].sum(axis=1))]

In [ ]:
# será esta a abordagem mais acertada, isto é, assumir que todas as compras deste cliente foram "OfferPurchases"?

df.loc[(df['NumOfferPurchases'] > df[['NumAppPurchases','NumTakeAwayPurchases','NumStorePurchases']].sum(axis=1)),'NumOfferPurchases'] = df['NumAppPurchases'] + df['NumTakeAwayPurchases'] + df['NumStorePurchases']

In [ ]:
df[(df['NumOfferPurchases'] > df[['NumAppPurchases','NumTakeAwayPurchases','NumStorePurchases']].sum(axis=1))] # confirmação do ajuste

## Data Review

Ver a dataframe no seu estado final
Drop: Id, name, birthyear, date_adherence, total_kids, mntTotal

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df_train= df.copy()
df_train.drop(['Name', 'Birthyear', 'Date_Adherence', 'Response_Cmp1', 'Response_Cmp2', 'Response_Cmp3', 'Response_Cmp4', 'Response_Cmp5',
               'Kid_Younger6', 'Children_6to18'], axis = 1, inplace = True)

In [ ]:
df_train.isna().sum()

# Feature Selection

As the problem is not a classification problem we will need to adapt our data so it can be used in classifiers. We will use Random Forest Classifiers as tools for feature selection, using MntTotal as the target variable, as our goal is to devise a marketing campaign that aims to increase sales

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
x= df_train.drop(['MntTotal', 'Mnt_pday_card', 'MntAdditionalRequests', 'MntDesserts', 'MntDrinks', 'MntEntries', 'MntMeat&Fish', 'MntVegan&Vegetarian'] , axis= 1)
y= df_train['MntTotal']
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=1)

y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

In [ ]:
scalerx = MinMaxScaler()
scalery = MinMaxScaler()

scalerx = scalerx.fit(x_train)
scalery = scalery.fit(y_train)

x_train = pd.DataFrame(scalerx.transform(x_train), columns= x.columns)
x_test = pd.DataFrame(scalerx.transform(x_test), columns= x.columns)

y_train = pd.DataFrame(scalery.transform(y_train))
y_test = pd.DataFrame(scalery.transform(y_test))
#df_train.describe()

In [ ]:
RFR = RandomForestRegressor(n_estimators=100, max_depth=5)
RFR.fit(x_train, y_train)

In [ ]:
importances = pd.Series(RFR.feature_importances_, index= x_train.columns)
importances

In [ ]:
threshold = importances.median()
selected_features = x_train.reset_index(drop=True).loc[:, importances >= threshold]
print(selected_features.columns.to_series().reset_index(drop=True).to_string(index=False))

In [ ]:
new_RFR = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=1)
new_RFR.fit(selected_features, y_train)

In [ ]:
x_test= x_test[selected_features.columns]
y_pred = new_RFR.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE: ", mse)

## Data scaling
min max: income, recency, mnt..., purchases ..., age, daysasClient, mnt per ...

In [ ]:
scaler = MinMaxScaler()
df_train = pd.DataFrame(scaler.fit_transform(df_train))
#df_train.describe()

## PCA


A implementação do pca acima pareceu me estranha. deixo aqui outra e quando reunirmos vemos

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
df_train2= df_train.copy()
pca.fit(df_train2)
var= pca.explained_variance_ratio_
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

plt.title("PCA Variance against num of Componmnets")
plt.ylabel("Variance %")
plt.xlabel("Number of componments")
l = plt.axhline(80, color="red")

plt.plot(var1)
plt.grid()

In [ ]:
pca = PCA(n_components=7)
pca_train=pca.fit_transform(df_train2)
pca_train

In [ ]:
df_train

7 variaveis explicam ~80% da variancia

## Loading Scores for each PC

##  Correlação entre PC's e as variáveis originais

## Model Train

In [ ]:
from sklearn.cluster import KMeans

kmeans= KMeans(n_clusters = 7, max_iter =10000, random_state= 1)
kmeans.fit(pca_train)
pca_train_label = kmeans.labels_
pca_train_label = pd.DataFrame(pca_train_label)

In [ ]:
df.reset_index(inplace=True)
pca_train_label.reset_index(inplace=True)
df_final=pd.concat([df,pca_train_label],axis=1)
df_final.set_index('CustomerID',inplace=True)
df_final.drop(['index'],axis=1,inplace=True)
df_final= df_final.rename(columns={0:'Cluster'})

In [ ]:
kmeans2= KMeans(n_clusters = 7, max_iter = 10000, random_state = 1)
kmeans2.fit(df_train)
df_train_label = kmeans2.labels_

In [ ]:
dfTF= pd.DataFrame()
dfTF['Compare'] = pca_train_label== df_train_label
#pd.DataFrame(dfTF.groupby('Compare'))
dfTF.groupby(['Compare']).size().reset_index(name='counts')

### 3.2.2. Misclassifications

### 3.2.3. Incoherencies

> 3.2.3.1 Clients that spent money but never had a registered Purchase

In [ ]:
df[(df[['MntMeat&Fish', 'MntEntries',
       'MntVegan&Vegetarian', 'MntDrinks', 'MntDesserts',
       'MntAdditionalRequests']].sum(axis = 1) >= 0) & (df[['NumOfferPurchases', 'NumAppPurchases',
       'NumTakeAwayPurchases', 'NumStorePurchases']].sum(axis = 1) <= 0)].shape

### 3.2.4. Binning

### 3.2.5. Reclassification

### 3.2.6. Power Transform

## 3.3. Data Reduction

### 3.3.1. Multicollinearity - Check correlation

### 3.3.2. Unary Variables

### 3.3.3. Variables with a high percentage of missing values

## 3.2. Back to Data Transformation

### 3.2.7. Apply ordinal encoding and create Dummy variables

### 3.2.8. Scaling

## 4. Models

### 4.1. K-Means

In [ ]:
# variavel modelo visao monetary: 
        #  Mnt_pday_card, has_Kids, Income, age_bins, 'Graduation', 'HighSchool', 'Master', 'PhD', 'Gender'

In [ ]:
df_value = df[["Mnt_pday_card", "has_Kids", "Income", "Age", 'Graduation', 'HighSchool', 'Master', 'PhD', 'Gender', "Recency" ]].copy()

In [ ]:
ks = range(1,11)
inertias = []


for k in ks:
    model = KMeans(n_clusters = k).fit(df_value)
    inertias.append(model.inertia_)

In [ ]:
# Plot ks (x-axis) vs inertias (y-axis) using plt.plot(). 
plt.plot(ks, inertias)

# define the label for the x axis as 'number of clusters' using matplotlib.pyplot.xlabel
plt.xlabel('number of clusters')
# define the label for the y axis as 'inertia' using matplotlib.pyplot.ylabel
plt.ylabel('inertia')
# define the ticks on the x axis using the values of ks
plt.xticks(ks)
# call plt.show()
plt.show()

In [ ]:
df_value_k2 = df_value.copy()
model_k2 = sk.cluster.KMeans(n_clusters = 2, random_state = 1).fit(df_value_k2)

In [ ]:
df_value_k2["label"] = model_k2.labels_

In [ ]:
df_value_k2["label"].value_counts()

In [ ]:
df_value_k2.groupby(["label"]).mean().transpose()

In [ ]:
df_value_k3 = df_value.copy()
model_k3 = sk.cluster.KMeans(n_clusters = 3, random_state = 2).fit(df_value_k3)

In [ ]:
df_value_k3["label"] = model_k3.labels_

In [ ]:
df_value_k3["label"].value_counts()

In [ ]:
df_value_k3.groupby(["label"]).describe()

Measuring distances between clusters

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

dists_k2 = euclidean_distances(model_k2.cluster_centers_)
dists_k2

In [ ]:
dists_k3 = euclidean_distances(model_k3.cluster_centers_)
dists_k3

### Visualization

In [ ]:
cluster0 = df_value_k2[df_value_k2.label == 0]
cluster1 = df_value_k2[df_value_k2.label == 1]


In [ ]:
sns.histplot(cluster0['Mnt_pday_card'],color='red',label='Cluster 0', bins = 20)
sns.histplot(cluster1['Mnt_pday_card'],color='yellow',label='Cluster 1', bins = 20)
plt.legend()

In [ ]:
cluster0 = df_value_k3[df_value_k3.label == 0]
cluster1 = df_value_k3[df_value_k3.label == 1]
cluster2 = df_value_k3[df_value_k3.label == 2]

In [ ]:
sns.histplot(cluster0['Mnt_pday_card'],color='red',label='Cluster 0', bins = 20)
sns.histplot(cluster1['Mnt_pday_card'],color='yellow',label='Cluster 1', bins = 20)
sns.histplot(cluster2['Mnt_pday_card'],color='green',label='Cluster 2', bins = 20)
plt.legend()

In [ ]:
sns.histplot(cluster0['Income'],color='red',label='Cluster 0', bins = 20)
sns.histplot(cluster1['Income'],color='yellow',label='Cluster 1', bins = 20)
sns.histplot(cluster2['Income'],color='green',label='Cluster 2', bins = 20)
plt.legend()

In [ ]:
# variavel modelo visao customer behaviour: NumOfferPurchases', 'NumAppPurchases',
       # 'NumTakeAwayPurchases', 'NumStorePurchases', 'NumAppVisitsMonth',
       # 'Complain', 'Gender', 'Income', 'Age', 'Graduation', 'HighSchool', 'Master', 'PhD', 'Married', 'Single',
       # 'Widow'